In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from  tqdm import  tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-04-19 11:47:38.523809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745063258.751773      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745063258.822643      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/cluster2model/tensorflow2/default/1/cluster2.weights.h5
/kaggle/input/ps-dataset/order_data_last_six_month.xlsx - Worksheet.csv
/kaggle/input/ps-dataset/associated_order_item_data_last_six_month.xlsx - Worksheet.csv


In [2]:
link='/kaggle/input/ps-dataset/order_data_last_six_month.xlsx - Worksheet.csv'

In [3]:
def fill_missing_predictions(all_customer_ids, pred_df, num_days=14):
    set_of_missing=list(set(all_customer_ids)-set(pred_df['customer_id']))
    empty_df=pd.concat([pd.Series(set_of_missing,name='customer_id'),pd.DataFrame(np.zeros(shape=(len(set_of_missing),num_days)))],axis=1)
    final_df=pd.concat([pred_df,empty_df],axis=0)
    final_df.sort_values(by='customer_id',inplace=True)
    return final_df

In [4]:
def inference(link):
    SEQ_LEN = 30
    PRED_HORIZON = 14
    df=pd.read_csv(link)
    all_customers=df['customer_id'].unique()
    df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=True)

    max_date = df['order_date'].max()
    customer_behavior=pd.DataFrame(df['customer_id'].unique(),columns=['customer_id'])
    
    for idx in df['customer_id'].unique():
        customer_data = df[df['customer_id'] == idx]
    
        total_orders = customer_data['order_id'].nunique()
        total_order_days = customer_data['order_date'].nunique()
    
        average_orders_placed = total_orders / total_order_days if total_order_days > 0 else 0
        total_order_value = customer_data['net_order_amount'].sum()
        average_order_value = total_order_value / total_orders if total_orders > 0 else 0

    # Calculate average order gap days
        order_dates = customer_data['order_date'].sort_values()
        order_gaps = order_dates.diff().dt.days.dropna()
        avg_order_gap_days = order_gaps.mean() if not order_gaps.empty else 192

    # Order consistency score (1 / std of gaps)
    # Assign to customer_behavior DataFrame
        customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Total Orders'] = total_orders
        customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Average Order Gap Days'] = avg_order_gap_days
    zero_people=customer_behavior[customer_behavior['Total Orders']<=4]['customer_id']
    cluster_1_people=customer_behavior[((customer_behavior['Total Orders']<=25) & (customer_behavior['Total Orders']>4) & (customer_behavior['Average Order Gap Days']>=14))]['customer_id']
    cluster_2_people = customer_behavior['customer_id'][~customer_behavior['customer_id'].isin(cluster_1_people) & ~customer_behavior['customer_id'].isin(zero_people)]

    train_data=df[df['customer_id'].isin(cluster_2_people)]
    train_data.drop(['order_number','customer_name','poc_name','poc_id','amount','profit','order_status','warehouse_id'],axis=1,inplace=True)
    df = train_data.copy()
    df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

# group & aggregate:
    daily_df = df.groupby(['customer_id','order_date']).agg(
        daily_order_count = ('order_id',       'count'),
        discount          = ('discount',       'sum'),
        net_order_amount  = ('net_order_amount','sum'),
        warehouse_name    = ('warehouse_name', 'first')  # or use mode if you prefer
    ).reset_index()
    daily_df_dummy=pd.get_dummies(daily_df,columns=['warehouse_name'],dtype='int')
    df = daily_df_dummy.copy()
    df['order_date'] = pd.to_datetime(df['order_date'])

# Add 'day_of_week' column
    df['day_of_week'] = df['order_date'].dt.dayofweek  # Monday=0, Sunday=6

# One-hot encode 'day_of_week'
    df = pd.get_dummies(df, columns=['day_of_week'], prefix='dow',dtype='int')



#---------#

    df = df[df['order_date'] >= max_date - pd.Timedelta(days=29)]

    pred_cust_id=df['customer_id'].unique()
# Identify feature columns (exclude ID/date)
    feature_cols = [c for c in df.columns 
                    if c not in ('customer_id', 'order_date')]

# Containers for sequence data
    X = []

    for cust_id, group in tqdm(df.groupby('customer_id'), total=df['customer_id'].nunique()):
        group = group.set_index('order_date').sort_index()
        full_idx = pd.date_range(group.index.min(), group.index.max(), freq='D')
        group = group.reindex(full_idx).fillna(0)

        group['customer_id'] = cust_id
        group['date'] = group.index

        data = group[feature_cols].values

        if len(data) >= SEQ_LEN:
            X.append(data[-SEQ_LEN:, :])  # only last 30 days
        else:
        # pad at beginning with zeros if less than 30
            padded = np.zeros((SEQ_LEN, data.shape[1]))
            padded[-len(data):, :] = data
            X.append(padded)

    X = np.stack(X)
    NUM_FEATURES = len(feature_cols)
    model_best = Sequential([
    # 1st LSTM layer, returns sequences so we can stack another LSTM
        LSTM(128, input_shape=(SEQ_LEN, NUM_FEATURES), return_sequences=True),
        Dropout(0.2),

    # 2nd LSTM layer
        LSTM(64, return_sequences=False),
        Dropout(0.2),

    # Dense “bottleneck” to learn combined features
        Dense(32, activation='relu'),
        Dropout(0.1),

    # Final output: 14 sigmoid neurons, one per future day
        Dense(PRED_HORIZON, activation='sigmoid')
    ])


# Important: you need to compile it again
    model_best.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Load the best saved weights
    model_best.load_weights('/kaggle/input/cluster2model/tensorflow2/default/1/cluster2.weights.h5')
    pred=model_best.predict(X)
    #pred_bin= (pred > 0.5).astype(int)   
    prob=pd.concat([pd.Series(pred_cust_id,name='customer_id'),pd.DataFrame(pred)],axis=1)
    
    return fill_missing_predictions(all_customers,prob)

In [5]:
pred=inference(link)

/tmp/ipykernel_13/2009342782.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(['order_number','customer_name','poc_name','poc_id','amount','profit','order_status','warehouse_id'],axis=1,inplace=True)
100%|██████████| 1565/1565 [00:03<00:00, 480.49it/s]
2025-04-19 11:48:03.840963: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:7

49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step


In [6]:
pred

,customer_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,0.294163,0.308527,0.315218,0.32275,0.321471,0.326835,0.325123,0.324396,0.319337,0.322451,0.317228,0.315706,0.314408,0.315198
2,4,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5,0.228650,0.233207,0.233014,0.23641,0.236311,0.237720,0.238132,0.240686,0.238223,0.239868,0.240317,0.244029,0.243465,0.250803
4,9,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,12,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,9349,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
639,9350,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
640,9352,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
641,9358,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
pred.to_csv("final_14_day_order_proba.csv",index=False)